In [ ]:
#source: this notebook was adapted from https://www.philschmid.de/fine-tune-a-non-english-gpt-2-model-with-huggingface


!pip install transformers

     |████████████████████████████████| 1.3MB 11.2MB/s 
     |████████████████████████████████| 890kB 50.3MB/s 
     |████████████████████████████████| 1.1MB 47.1MB/s 
     |████████████████████████████████| 2.9MB 50.8MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893257 sha256=d0081a121908b59b422a97f136d60cf8d17b50e0742aeecc7ed2a8f95aeb7b1f
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


In [ ]:
!nvidia-smi

Wed Nov 25 00:23:16 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 455.38       Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   53C    P8    10W /  70W |      0MiB / 15079MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
#upload files to your colab environment
from google.colab import files
uploaded = files.upload()

Saving text_adventures.txt to text_adventures.txt


After we uploaded the file with use `unzip` to extract the recipes.json. 

In [ ]:
import re
import json
from sklearn.model_selection import train_test_split


file = open('text_adventures.txt', encoding='utf8')
contents = file.read()
split_contents = contents.split('<|endoftext|>')
split_contents_string = ''
for content in split_contents:
  split_contents_string += content
split_contents = split_contents_string.split('<|startoftext|>')

def build_text_files(data, dest_path):
    data_string = ''
    for text in data:
        data_string += text
    f = open(dest_path, 'w')
    f.write(data_string)

train, test = train_test_split(split_contents, test_size=0.15)

build_text_files(train,'train_dataset.txt')
build_text_files(test,'test_dataset.txt')

print("Train dataset length: "+str(len(train)))
print("Test dataset length: "+ str(len(test)))

Train dataset length: 2155
Test dataset length: 381


In [ ]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("gpt2")

train_path = 'train_dataset.txt'
test_path = 'test_dataset.txt'

In [ ]:
from transformers import TextDataset,DataCollatorForLanguageModeling

def load_dataset(train_path,test_path,tokenizer):
    train_dataset = TextDataset(
          tokenizer=tokenizer,
          file_path=train_path,
          block_size=128)
     
    test_dataset = TextDataset(
          tokenizer=tokenizer,
          file_path=test_path,
          block_size=128)   
    
    data_collator = DataCollatorForLanguageModeling(
        tokenizer=tokenizer, mlm=False,
    )
    return train_dataset,test_dataset,data_collator

train_dataset,test_dataset,data_collator = load_dataset(train_path,test_path,tokenizer)

/usr/local/lib/python3.6/dist-packages/transformers/data/datasets/language_modeling.py:44: FutureWarning: This dataset will be removed from the library soon, preprocessing should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/master/examples/language-modeling/run_mlm.py
  FutureWarning,


# Initialize `Trainer` with `TrainingArguments` and GPT-2 model

The [Trainer](https://huggingface.co/transformers/main_classes/trainer.html#transformers.Trainer) class provides an API for feature-complete training. It is used in most of the [example scripts](https://huggingface.co/transformers/examples.html) from Huggingface. Before we can instantiate our `Trainer` we need to download our GPT-2 model and create a [TrainingArguments](https://huggingface.co/transformers/main_classes/trainer.html#transformers.TrainingArguments) to access all the points of customization during training. In the `TrainingArguments`, we can define the Hyperparameters we are going to use in the training process like our `learning_rate`, `num_train_epochs`, or  `per_device_train_batch_size`. A complete list can you find [here](https://huggingface.co/transformers/main_classes/trainer.html#trainingarguments).

In [ ]:
from transformers import Trainer, TrainingArguments,AutoModelWithLMHead

model = AutoModelWithLMHead.from_pretrained("gpt2")


training_args = TrainingArguments(
    output_dir="./gpt2", #The output directory
    overwrite_output_dir=True, #overwrite the content of the output directory
    num_train_epochs=3, # number of training epochs
    per_device_train_batch_size=32, # batch size for training
    per_device_eval_batch_size=64,  # batch size for evaluation
    eval_steps = 400, # Number of update steps between two evaluations.
    save_steps=800, # after # steps model is saved 
    warmup_steps=500,# number of warmup steps for learning rate scheduler
    )


trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    prediction_loss_only=True,
)

/usr/local/lib/python3.6/dist-packages/transformers/modeling_auto.py:837: FutureWarning: The class `AutoModelWithLMHead` is deprecated and will be removed in a future version. Please use `AutoModelForCausalLM` for causal language models, `AutoModelForMaskedLM` for masked language models and `AutoModelForSeq2SeqLM` for encoder-decoder models.
  FutureWarning,


/usr/local/lib/python3.6/dist-packages/transformers/trainer.py:281: FutureWarning: Passing `prediction_loss_only` as a keyword argument is deprecated and won't be possible in a future version. Use `args.prediction_loss_only` instead. Setting `args.prediction_loss_only=True
  FutureWarning,


# Train and save the model

To train the model we can simply run `Trainer.train()`.

In [ ]:
import time

starttime = time.time()
trainer.train()
print('Time taken to fine tune GPT2 is {}'.format(time.time() - starttime))

Step,Training Loss
500,3.375289
1000,3.138445
1500,3.055061
2000,2.987678
2500,2.962112
3000,2.922194
3500,2.892709
4000,2.891039


Time taken to fine tune GPT2 is 4853.940411090851


After training is done you can save the model by calling `save_model()`. This will save the trained model to our `output_dir` from our `TrainingArguments`.

In [ ]:
trainer.save_model()

# Test the model

To test the model we are going to use another [highlight of the transformers library](https://huggingface.co/transformers/main_classes/pipelines.html?highlight=pipelines) called `pipeline`. [Pipelines](https://huggingface.co/transformers/main_classes/pipelines.html?highlight=pipelines) are objects that offer a simple API dedicated to several tasks, among others also `text-generation`

In [ ]:
from transformers import pipeline

generator_pipeline = pipeline('text-generation', model='./gpt2', tokenizer='gpt2',config={'max_length':3200})

result = generator_pipeline('Luke is the son of Vader.')[0]['generated_text']

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


In [ ]:
print(result)

Luke is the son of Vader. You’re the only one with the same dad in real life and it’s obvious you feel the same way. It’s as if Tulla sees your eyes, but she’s


In [ ]:
result2 = generator_pipeline("You wonder if it’s time to call on his son, it’s time to take him back into his own realm, maybe you need to")
print(result2)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


[{'generated_text': 'You wonder if it’s time to call on his son, it’s time to take him back into his own realm, maybe you need to try this out.\n\nYou turn to the pair of the wolflings and'}]


In [ ]:
result3 = generator_pipeline("The story is set in a beautiful castle")[0]['generated_text']
print(result3)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


The story is set in a beautiful castle. The Emperor has chosen it as his capital, but there hasn't been much progress in building the palace, and so the Council is proposing that the only way to get there is to sail south through the jung


In [ ]:
result4 = generator_pipeline("Prince charming lived in the castle.")[0]['generated_text']
print(result4)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Prince charming lived in the castle. When you arrived you visited the castle.” Mistress says.
“Heh, well…” you say getting into your own seat.
“And what did you want?”



In [ ]:
result5 = generator_pipeline("Your nemesis found you under the bed.")[0]['generated_text']
print(result5)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Your nemesis found you under the bed.

“Hey Suzy, just get on back on and start fucking out.” You say to your sister.
“Yeah, I’ll probably get on my Harley and


In [ ]:
result6 = generator_pipeline('Harry looked at Hermione')[0]['generated_text']
print(result6)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Harry looked at Hermione’s hand and saw that she didn’t say anything.

“Okay, let’s get this straight. Not killing anyone isn’t going to make the Empire any better, now what


In [ ]:
result7 = generator_pipeline('Your nemesis found you under the bed. As you try and stand up, a hand grabs at your face. \n\n "Hey, hold on there, OK!"')[0]['generated_text']
print(result7)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Your nemesis found you under the bed. As you try and stand up, a hand grabs at your face. 

 "Hey, hold on there, OK!"  Its a desperate desperate sound, but the other hand stops it before it can


In [ ]:
test[2]

'You sprint towards the lamp, your rifle held tightly in your hands as you run towards it. Draven charges ahead with a howl, kicking your leg away as you collapse onto the ground, laughing.\n\n\n"Look at you try to run! Where are you trying to run to, little one?" he asks, before his eyes flicker to Dasha who goes for the door.\n\n\nDraven runs ahead with an animalistic howl, slamming her into the door. He presses the sword to her throat, before throwing into the air and catching it once more.\n\n\n"Ohh... this is everything! Savor the hunt!" he says. "Go! Try escape again! Or are you in a fighting mood?"\n> You try for the lamp again\nYou run towards the lamp, as Draven realizes what you\'re doing, and charges forward. He grabs the lamp, opening it with the flick of his wrist and blowing out the candle. \n\n\n"Nice attempt," he says. "Now run."\n\n\nYou turn and run, as Draven follows. He swings his sword as you run , and the next thing you know, you feel the cold bite of steel on you

In [ ]:
from google.colab import files
files.download('test_dataset.txt')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
files.download('/content/gpt2/config.json')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
files.download('/content/gpt2/pytorch_model.bin')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
files.download('/content/gpt2/training_args.bin')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
files.download('/content/cached_lm_GPT2Tokenizer_128_test_dataset.txt')
files.download('/content/cached_lm_GPT2Tokenizer_128_test_dataset.txt.lock')
files.download('/content/cached_lm_GPT2Tokenizer_128_train_dataset.txt')
files.download('/content/cached_lm_GPT2Tokenizer_128_train_dataset.txt.lock')
files.download('/content/train_dataset.txt')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
files.download('/content/runs/Nov21_19-01-12_7fab1c6deb81/1605985286.3252249/events.out.tfevents.1605985286.7fab1c6deb81.56.1')
files.download('/content/runs/Nov21_19-01-12_7fab1c6deb81/events.out.tfevents.1605985286.7fab1c6deb81.56.0')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>